In [48]:
import json
import pandas as pd
import numpy as np

import os

In [49]:

import json
import pandas as pd
import numpy as np

def load_data(path):
    with open(path, 'r') as file:
        data = json.load(file)

    parsed_data = {}
    for key, value in data.items():
        # Check if the value is a list
        if isinstance(value, list):
            try:
                # Attempt to convert to a NumPy array
                subfield = np.array(value)

                if subfield.ndim == 1:  # 1D array
                    parsed_data[key] = pd.Series(value)
                elif subfield.ndim == 2:  # 2D array
                    parsed_data[key] = pd.DataFrame(value)
                else:  # Higher-dimensional array
                    parsed_data[key] = {i: pd.DataFrame(subarray) for i, subarray in enumerate(subfield)}
            except ValueError:
                # If conversion fails, handle as a dictionary of DataFrames
                parsed_data[key] = {i: pd.DataFrame(subarray) for i, subarray in enumerate(value) if isinstance(subarray, list)}
        else:
            # For non-list data, store it as-is
            parsed_data[key] = value

    return parsed_data

In [50]:
# Path to your JSON file
# Define the directories
results_dir = "results"   
results_csv_dir = "results_csv" 

# Create the results_csv directory if it doesn't exist
os.makedirs(results_csv_dir, exist_ok=True)
# Process each JSON file in the results directory
for filename in os.listdir(results_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(results_dir, filename)
        
        # Parse the JSON data
        parsed_data = load_data(file_path)

        # Get the subfolder name (without .json)
        subfolder_name = os.path.splitext(filename)[0]
        subfolder_name = subfolder_name[subfolder_name.find('_')+1:] if '_' in subfolder_name else subfolder_name
        subfolder_path = os.path.join(results_csv_dir, subfolder_name)
        
        # Create the subfolder
        os.makedirs(subfolder_path, exist_ok=True)

        # Save all Dw matrices to CSV
        if 'all_Dw' in parsed_data:
            for i, dw_matrix in parsed_data['all_Dw'].items():
                dw_csv_path = os.path.join(subfolder_path, f"Dw_{i}.csv")
                dw_matrix.to_csv(dw_csv_path, index=False, header=False)  # Exclude index and header

        # Save all Dkl matrices to CSV
        if 'all_Dkl' in parsed_data:
            for i, dkl_matrix in parsed_data['all_Dkl'].items():
                dkl_csv_path = os.path.join(subfolder_path, f"Dkl_{i}.csv")
                dkl_matrix.to_csv(dkl_csv_path, index=False, header=False)  # Exclude index and header
